<a href="https://colab.research.google.com/github/DDubrovina/3_Task/blob/main/3_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Третья часть

Представь, что в рамках разработки сервиса понадобилось заказать разметку у внешней команды разметчиков. Команда разметки подготовила массив данных для обучения модели (файл public_protokols.tar.gz).

Что это за разметка? Это документы типа «Протокол» в формате html, которые получены после распознавания сканов исходных документов. К документам прилагается таблица с информацией по их разметке (файл markup.zip). В колонке «DocName» таблицы расположено название документа, чтобы отделить текст разных документов друг от друга. В колонке «Tokens» - разбитый на токены текст исходного документа.

Также для каждого токена в каждом документе представлена разметка по двум целям:

Выделение именованных сущностей, по сути классический NER (В имени колонки префикс «КЕ»);
Сегментация документа или разбиение на несколько семантически различных блоков (В имени колонки префикс «Blocks»);
Каждая цель разметки состоит из нескольких колонок, соответствующих разным сущностям. Сущности в рамках одной цели разметки разнесены по разным столбцам так как они могут как пересекаться и включаться одна в другую. Таким образом для каждого токена в тексте документа имеется множество меток. В общем случае сущность, в каждой из предлагаемых целей разметки, может состоять из произвольного числа токенов.

Если имя сущности имеет префикс «B-» это означает начало сущности; «I-» - середина или конец сущности. Не все сущности имеют префиксы, и если в какой-то колонке их нет, это означает, что нам не нужно различать одинаковые сущности следующие друг за другом.

Попробуй написать такой скрипт на Python (версии не ниже 3.6), с помощью которого можно быстро проверить предложенную разметку на «вшивость» (вдруг разметчики где-то схалтурили?) и откровенные ошибки.

Результат ждем в виде .ipynb-ноутбука (предпочтительно – выложенный на репозиторий в Github) или в виде ноутбука Google Colab (но это менее предпочтительный вариант).

##Загрузка и проверка файлов

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/Tasks/task3/public_protokols.tar/

/content/drive/My Drive/Tasks/task3/public_protokols.tar


In [ ]:
!ls

 180517_rus_minutes_No.37_ВОСА_16.05.2018.html	        protokol_2_vosa.html
'4. Выписка СД_26.04.2019 Распределение прибыли.html'  'protokol_gosa_30 06 2011.html'
 BOCA_protocol_n1_02_04_2012.html		        protokol_vosa_3_ot27_12_2011.html
 cf070284588b587d50022505f8c00285.html		       'report_itogi_18.01.2019(2).html'
 GOSA15_06_2012.html				        report_itogi_290618.html
 GOSA17_06_2014.html				        vbank_protocol25_vosa_070318.html
 gosa_27_06_2013.html				        vbank_protocol_vosa_070318.html
 itogi_golosovaniya.html			        vbank_protokol_2_vosa_2014.html
 itogi_vosa_070318.html				        vipiska_8.html
 markup.csv					        vosa_29092017.html
 protocol_31.05.2019.html			        voting_report_2019.html
'Protocol BOCA_18_06_2018.html'			       'Протокол ВОСА № 1 от 12_09_2017.html'
 protocol_gosa_18.01.2019.html			       'Протокол ВОСА № 1 от 18_10_2013.html'
 protocol_gosa_2029.06.2018.html		       'Протокол ВОСА № 2 от 19_12_2013.html'
 protocol_meeting_07032018.html	

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('markup.csv',
                        sep="\t",
                        encoding='utf8',
                        #names=my_cols,
                        #header=None,
                        engine="python", on_bad_lines='skip')

In [ ]:
df

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
0,Протокол ВОСА № 2 от 19_12_2013.html.txt,о,O,O,O,O,O,O,O,O,O
1,Протокол ВОСА № 2 от 19_12_2013.html.txt,',O,O,O,O,O,O,O,O,O
2,Протокол ВОСА № 2 от 19_12_2013.html.txt,«,O,O,O,O,O,O,O,O,O
3,Протокол ВОСА № 2 от 19_12_2013.html.txt,•,O,O,O,O,O,O,O,O,O
4,Протокол ВОСА № 2 от 19_12_2013.html.txt,ДВЭУК,O,O,O,O,O,O,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...
113950,GOSA17_06_2014.html.txt,Гепапов,B-ФизЛицо,O,O,O,O,10_Блок_Подписи,O,O,O
113951,GOSA17_06_2014.html.txt,К,I-ФизЛицо,O,O,O,O,10_Блок_Подписи,O,O,O
113952,GOSA17_06_2014.html.txt,.,I-ФизЛицо,O,O,O,O,10_Блок_Подписи,O,O,O
113953,GOSA17_06_2014.html.txt,В,I-ФизЛицо,O,O,O,O,10_Блок_Подписи,O,O,O


In [ ]:
df['DocName'].value_counts()

protocol_gosa_2029.06.2018.html.txt                        8682
gosa_27_06_2013.html.txt                                   8148
protocol_31.05.2019.html.txt                               7894
Протокол ГОСА_16_06_2017.html.txt                          6447
cf070284588b587d50022505f8c00285.html.txt                  5932
Протокол ГОСА_2016.html.txt                                5537
GOSA17_06_2014.html.txt                                    5531
180517_rus_minutes_No.37_ВОСА_16.05.2018.html.txt          5481
Protocol BOCA_18_06_2018.html.txt                          5279
vosa_29092017.html.txt                                     4853
Протокол ВОСА № 1 от 18_10_2013.html.txt                   4682
protokol_2_vosa.html.txt                                   3667
voting_report_2019.html.txt                                3279
report_itogi_290618.html.txt                               3163
GOSA15_06_2012.html.txt                                    3090
protokol_gosa_30 06 2011.html.txt       

Сравниваем разметку и исходные файлы. Не хватает -

##Проверка именнованных сущностей (KE)

Проводим анализ содержания разметки каждого столбца.

In [ ]:
df['KE_0'].value_counts()

O                      85445
I-НЮЛ                   7233
I-ФизЛицо               3637
I-Дата                  3083
I-Орган_ОСА             2771
I-Сумма                 1911
B-НЮЛ                   1798
B-ФизЛицо               1407
B-Орган_ОСА             1315
B-Дата                   936
I-Орган_СД               847
B-Орган_СД               814
I-Орган_Отсутствует      511
B-Орган_Отсутствует      490
I-Время                  461
I-ГосОрган               300
B-Сумма                  265
I-Орган_ЕИО              179
B-Орган_ЕИО              172
B-Время                  164
B-ГосОрган               145
B-Орган_ОСУ               36
B-Орган_КИО               32
I-Орган_ОСУ                3
Name: KE_0, dtype: int64

In [ ]:
df['KE_1'].value_counts()

O                         103508
I-НЮЛ-Наименование          4782
B-НЮЛ-Наименование          1586
B-НЮЛ-ОПФ                   1566
B-Орган-Форма                783
I-НЮЛ-ОПФ                    669
I-Сумма-сумма_прописью       475
B-Сумма-валюта               330
B-Сумма-сумма_прописью       112
I-Сумма-валюта                83
B-Сумма-оператор              35
I-Сумма-оператор              26
Name: KE_1, dtype: int64

In [ ]:
df['KE_2'].value_counts()

O              112049
I-Должность       962
B-Должность       944
Name: KE_2, dtype: int64

In [ ]:
df['KE_3'].value_counts()

O                           89788
I-Цифры_Номер порядковый     6487
I-Цифры_Кол-во               5993
B-Цифры_Номер порядковый     2462
I-Адрес                      2306
B-Цифры_Кол-во               1852
I-Цифры_Номер                1786
I-Цифры_Процент              1700
B-Цифры_Процент               745
B-Цифры_Номер                 418
I-Цифры_Длительность          166
B-Адрес                       157
B-Цифры_Длительность           56
B-Цифры_Иная цифра             21
I-Цифры_Иная цифра             18
Name: KE_3, dtype: int64

In [ ]:
df['KE_4'].value_counts()

O                            93745
I-Цифры-цифры_номером         5849
B-Цифры-цифры_номером         5064
I-Цифры-БазаОпред             2346
I-Цифры-цифры_прописью        1805
B-Цифры-единица_измерения     1489
B-Цифры-БазаОпред             1146
B-Цифры-цифры_прописью         861
I-Адрес-Дом                    387
I-Адрес-Улица                  213
I-Адрес-СубъектРФ              193
B-Адрес-СубъектРФ              125
B-Адрес-Дом                    114
I-Цифры-единица_измерения      114
B-Адрес-Улица                  111
I-Адрес-НасПункт               110
B-Цифры-оператор               105
B-Адрес-Индекс                  69
B-Адрес-НасПункт                56
I-Цифры-оператор                53
Name: KE_4, dtype: int64

###Проверка на несовпадение категорий "ФизЛицо" и "Должность"

In [ ]:
df[(df.KE_0.str.contains('ФизЛицо')) & (df.KE_2.str.contains('Должность'))]

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
1251,Протокол ВОСА № 2 от 19_12_2013.html.txt,Дальнего,B-ФизЛицо,O,I-Должность,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,O
1252,Протокол ВОСА № 2 от 19_12_2013.html.txt,Востока,I-ФизЛицо,O,I-Должность,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,O
1462,Протокол ВОСА № 2 от 19_12_2013.html.txt,Дальнего,B-ФизЛицо,O,I-Должность,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Голосовали
1463,Протокол ВОСА № 2 от 19_12_2013.html.txt,Востока,I-ФизЛицо,O,I-Должность,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Голосовали
58430,gosa_27_06_2013.html.txt,Дальнего,B-ФизЛицо,O,I-Должность,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Голосовали
58431,gosa_27_06_2013.html.txt,Востока,I-ФизЛицо,O,I-Должность,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Голосовали


###Проверка на наличие разделения одной должности на несколько сущностей
Например, вместо единой сущности "советник министра", две - "советник" и "министра"

In [ ]:
i_old = -1
for i, row in df[(df.KE_2 == 'B-Должность')].iterrows():
  if i - i_old == 1:
    print(df.loc[i_old:i+1])
  else:
    i_old = i

In [ ]:
pd.set_option('display.max_columns', 500)

###Проверка отнесения чисел к содержательной сущности (кроме "0")

In [ ]:
df[(df['Tokens'].str.isdigit()) & ((df['KE_3'] == 'O') | (df['KE_4'] == 'O'))]

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
7,Протокол ВОСА № 2 от 19_12_2013.html.txt,2,O,O,O,I-Цифры_Номер,O,1_Блок_Заголовок,O,O,O
97,Протокол ВОСА № 2 от 19_12_2013.html.txt,19,B-Дата,O,O,O,O,1_Блок_Заголовок,O,O,O
99,Протокол ВОСА № 2 от 19_12_2013.html.txt,2013,I-Дата,O,O,O,O,1_Блок_Заголовок,O,O,O
138,Протокол ВОСА № 2 от 19_12_2013.html.txt,13,B-Время,O,O,O,O,1_Блок_Заголовок,O,O,O
140,Протокол ВОСА № 2 от 19_12_2013.html.txt,30,I-Время,O,O,O,O,1_Блок_Заголовок,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...
113681,GOSA17_06_2014.html.txt,2014,I-Дата,O,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Выслушали_Обсудили
113722,GOSA17_06_2014.html.txt,08,B-Дата,O,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Выслушали_Обсудили
113724,GOSA17_06_2014.html.txt,05,I-Дата,O,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Выслушали_Обсудили
113726,GOSA17_06_2014.html.txt,2014,I-Дата,O,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Выслушали_Обсудили


In [ ]:
df[(df['Tokens'].str.isdigit()) & (df.KE_0.str.contains('Дата')) & ((df.KE_3.str.contains('Цифры')) | (df.KE_4.str.contains('Цифры')))]

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
2667,Протокол ВОСА № 2 от 19_12_2013.html.txt,23,B-Дата,O,O,B-Цифры_Номер порядковый,B-Цифры-цифры_номером,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Выслушали_Обсудили
2669,Протокол ВОСА № 2 от 19_12_2013.html.txt,1,I-Дата,O,O,I-Цифры_Номер порядковый,I-Цифры-цифры_номером,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Выслушали_Обсудили
2670,Протокол ВОСА № 2 от 19_12_2013.html.txt,1,I-Дата,O,O,I-Цифры_Номер порядковый,I-Цифры-цифры_номером,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Выслушали_Обсудили
4071,protocol_gosa_18.01.2019.html.txt,27,B-Дата,O,O,I-Цифры_Номер,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->КворумПоВопросу
4073,protocol_gosa_18.01.2019.html.txt,06,I-Дата,O,O,I-Цифры_Номер,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->КворумПоВопросу
4075,protocol_gosa_18.01.2019.html.txt,2002,I-Дата,O,O,I-Цифры_Номер,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->КворумПоВопросу
4771,protocol_gosa_18.01.2019.html.txt,27,B-Дата,O,O,I-Цифры_Номер,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->КворумПоВопросу
4773,protocol_gosa_18.01.2019.html.txt,06,I-Дата,O,O,I-Цифры_Номер,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->КворумПоВопросу
4775,protocol_gosa_18.01.2019.html.txt,2002,I-Дата,O,O,I-Цифры_Номер,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->КворумПоВопросу
6478,protocol_gosa_2029.06.2018.html.txt,27,B-Дата,O,O,I-Цифры_Номер,O,2_Блок_Участники,O,O,O


In [ ]:
df[(df['Tokens'].str.isdigit()) & (~(df.KE_3.str.contains('Цифры')) & ~(df.KE_3.str.contains('O')))].head(100)

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
60,Протокол ВОСА № 2 от 19_12_2013.html.txt,1,O,O,O,I-Адрес,I-Адрес-Дом,1_Блок_Заголовок,O,O,O
122,Протокол ВОСА № 2 от 19_12_2013.html.txt,1,O,O,O,I-Адрес,I-Адрес-Дом,1_Блок_Заголовок,O,O,O
2980,protocol_gosa_18.01.2019.html.txt,101000,O,O,O,I-Адрес,B-Адрес-Индекс,1_Блок_Заголовок,O,O,O
2990,protocol_gosa_18.01.2019.html.txt,7,O,O,O,I-Адрес,I-Адрес-Дом,1_Блок_Заголовок,O,O,O
2992,protocol_gosa_18.01.2019.html.txt,4,O,O,O,I-Адрес,I-Адрес-Дом,1_Блок_Заголовок,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...
28391,vbank_protocol_vosa_070318.html.txt,4,O,O,O,I-Адрес,I-Адрес-Дом,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили
28395,vbank_protocol_vosa_070318.html.txt,1,O,O,O,I-Адрес,I-Адрес-Дом,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили
28411,vbank_protocol_vosa_070318.html.txt,101990,O,O,O,I-Адрес,B-Адрес-Индекс,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили
28425,vbank_protocol_vosa_070318.html.txt,7,O,O,O,I-Адрес,I-Адрес-Дом,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили


###Код отсутствует

In [ ]:
df.loc[df['KE_0'] == "Орган_Отсутствует"]

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3


Знаки препинания, которые являются наиболее часто встречаемыми размеченными данными,  и кавычки не имеют критического значения при разметке.

In [ ]:
df[['Tokens', 'KE_1']].value_counts()

Tokens           KE_1     
.                O            7469
,                O            5172
в                O            2310
:                O            1800
по               O            1764
                              ... 
позиций          O               1
Рентабельность   O               1
Республиканское  I-НЮЛ-ОПФ       1
Риалов           O               1
►                O               1
Length: 5685, dtype: int64

In [ ]:
df_1 = df.loc[df['Tokens'] == "'"]

In [ ]:
df_2 = df.loc[(df['Tokens'] == "'") & (df['KE_0'] != "O")]

In [ ]:
df_2

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
33273,Протокол ВОСА № 1 от 18_10_2013.html.txt,',I-НЮЛ,I-НЮЛ-Наименование,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили
33297,Протокол ВОСА № 1 от 18_10_2013.html.txt,',I-НЮЛ,I-НЮЛ-Наименование,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили
43783,Protocol BOCA_18_06_2018.html.txt,',I-НЮЛ,I-НЮЛ-Наименование,O,O,O,O,O,O,O
93569,itogi_vosa_070318.html.txt,',I-НЮЛ,I-НЮЛ-Наименование,O,O,O,3_Блок_Повестка,O,3_1_ПовесткаДня,I-3_1_ПовесткаДня->Текст
94962,Protocol ВОСА_26_12_2018 № 2.html.txt,',I-НЮЛ,I-НЮЛ-Наименование,O,O,O,O,O,O,O
99157,report_itogi_290618.html.txt,',I-НЮЛ,I-НЮЛ-Наименование,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Голосовали
99158,report_itogi_290618.html.txt,',I-НЮЛ,I-НЮЛ-Наименование,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Голосовали


In [ ]:
df_3 = df.loc[(df['Tokens'] == "«") & (df['KE_0'] != "O")]
df_3

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
15,Протокол ВОСА № 2 от 19_12_2013.html.txt,«,I-НЮЛ,B-НЮЛ-Наименование,O,O,O,1_Блок_Заголовок,O,O,O
23,Протокол ВОСА № 2 от 19_12_2013.html.txt,«,I-НЮЛ,B-НЮЛ-Наименование,O,O,O,1_Блок_Заголовок,O,O,O
34,Протокол ВОСА № 2 от 19_12_2013.html.txt,«,I-НЮЛ,B-НЮЛ-Наименование,O,O,O,1_Блок_Заголовок,O,O,O
215,Протокол ВОСА № 2 от 19_12_2013.html.txt,«,I-НЮЛ,B-НЮЛ-Наименование,O,O,O,2_Блок_Участники,O,O,O
229,Протокол ВОСА № 2 от 19_12_2013.html.txt,«,I-НЮЛ,B-НЮЛ-Наименование,O,O,O,2_Блок_Участники,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...
113754,GOSA17_06_2014.html.txt,«,I-НЮЛ,B-НЮЛ-Наименование,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Вопрос_поставленный_...
113817,GOSA17_06_2014.html.txt,«,I-НЮЛ,B-НЮЛ-Наименование,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили
113911,GOSA17_06_2014.html.txt,«,I-НЮЛ,B-НЮЛ-Наименование,O,O,O,4_Блок_Обсуждение,O,O,O
113933,GOSA17_06_2014.html.txt,«,I-НЮЛ,B-НЮЛ-Наименование,O,O,O,10_Блок_Подписи,O,O,O


In [ ]:
df_4 = df.loc[(df['Tokens'] == "»") & (df['KE_0'] != "O")]
df_4

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
20,Протокол ВОСА № 2 от 19_12_2013.html.txt,»,I-НЮЛ,I-НЮЛ-Наименование,O,O,O,1_Блок_Заголовок,O,O,O
25,Протокол ВОСА № 2 от 19_12_2013.html.txt,»,I-НЮЛ,I-НЮЛ-Наименование,O,O,O,1_Блок_Заголовок,O,O,O
39,Протокол ВОСА № 2 от 19_12_2013.html.txt,»,I-НЮЛ,I-НЮЛ-Наименование,O,O,O,1_Блок_Заголовок,O,O,O
217,Протокол ВОСА № 2 от 19_12_2013.html.txt,»,I-НЮЛ,I-НЮЛ-Наименование,O,O,O,2_Блок_Участники,O,O,O
231,Протокол ВОСА № 2 от 19_12_2013.html.txt,»,I-НЮЛ,I-НЮЛ-Наименование,O,O,O,2_Блок_Участники,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...
113756,GOSA17_06_2014.html.txt,»,I-НЮЛ,I-НЮЛ-Наименование,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Вопрос_поставленный_...
113819,GOSA17_06_2014.html.txt,»,I-НЮЛ,I-НЮЛ-Наименование,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили
113913,GOSA17_06_2014.html.txt,»,I-НЮЛ,I-НЮЛ-Наименование,O,O,O,4_Блок_Обсуждение,O,O,O
113935,GOSA17_06_2014.html.txt,»,I-НЮЛ,I-НЮЛ-Наименование,O,O,O,10_Блок_Подписи,O,O,O


In [ ]:
df_5 = df.loc[(df['Tokens'] == "(") & (df['KE_0'] != "O")]
df_5

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
3347,protocol_gosa_18.01.2019.html.txt,(,I-НЮЛ,O,O,O,O,2_Блок_Участники,O,O,O
3528,protocol_gosa_18.01.2019.html.txt,(,I-НЮЛ,O,O,O,O,2_Блок_Участники,O,O,O
3564,protocol_gosa_18.01.2019.html.txt,(,I-НЮЛ,O,O,O,O,2_Блок_Участники,O,O,O
3589,protocol_gosa_18.01.2019.html.txt,(,I-НЮЛ,O,O,O,O,2_Блок_Участники,O,O,O
3615,protocol_gosa_18.01.2019.html.txt,(,I-НЮЛ,O,O,O,O,2_Блок_Участники,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...
107435,protokol_2_vosa.html.txt,(,I-НЮЛ,I-НЮЛ-Наименование,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили
107550,protokol_2_vosa.html.txt,(,I-Сумма,O,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили
107580,protokol_2_vosa.html.txt,(,I-Сумма,O,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили
107630,protokol_2_vosa.html.txt,(,I-Сумма,O,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили


In [ ]:
df_6 = df.loc[(df['Tokens'] == ")") & (df['KE_0'] != "O")]
df_6

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
3318,protocol_gosa_18.01.2019.html.txt,),I-НЮЛ,O,O,O,O,2_Блок_Участники,O,O,O
3775,protocol_gosa_18.01.2019.html.txt,),I-НЮЛ,O,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Выслушали_Обсудили
3842,protocol_gosa_18.01.2019.html.txt,),I-НЮЛ,O,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Выслушали_Обсудили
3934,protocol_gosa_18.01.2019.html.txt,),I-НЮЛ,O,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Вопрос_поставленный_...
4246,protocol_gosa_18.01.2019.html.txt,),I-НЮЛ,O,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили
...,...,...,...,...,...,...,...,...,...,...,...
107437,protokol_2_vosa.html.txt,),I-НЮЛ,I-НЮЛ-Наименование,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили
107564,protokol_2_vosa.html.txt,),I-Сумма,O,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили
107592,protokol_2_vosa.html.txt,),I-Сумма,O,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили
107644,protokol_2_vosa.html.txt,),I-Сумма,O,O,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Решили


##Проверка блоков (Blocks)

Рассмотрим документ protokol_2_vosa.html

In [ ]:
df_protokol = df.loc[(df['DocName'] == "protokol_2_vosa.html.txt")]
df_protokol

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
104757,protokol_2_vosa.html.txt,ДВЭУК,O,O,O,O,O,O,O,O,O
104758,protokol_2_vosa.html.txt,ПРОТОКОЛ,O,O,O,B-Цифры_Номер,B-Цифры-БазаОпред,1_Блок_Заголовок,O,O,O
104759,protokol_2_vosa.html.txt,№,O,O,O,I-Цифры_Номер,O,1_Блок_Заголовок,O,O,O
104760,protokol_2_vosa.html.txt,2,O,O,O,I-Цифры_Номер,O,1_Блок_Заголовок,O,O,O
104761,protokol_2_vosa.html.txt,внеочередного,B-Орган_ОСА,B-Орган-Форма,O,O,O,1_Блок_Заголовок,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...
108419,protokol_2_vosa.html.txt,собрания,I-Орган_ОСА,O,O,O,O,10_Блок_Подписи,O,O,O
108420,protokol_2_vosa.html.txt,акционеров,I-Орган_ОСА,O,O,O,O,10_Блок_Подписи,O,O,O
108421,protokol_2_vosa.html.txt,ОАО,B-НЮЛ,B-НЮЛ-ОПФ,O,O,O,10_Блок_Подписи,O,O,O
108422,protokol_2_vosa.html.txt,«,I-НЮЛ,B-НЮЛ-Наименование,O,O,O,10_Блок_Подписи,O,O,O


По идее, первый блок нужно закончить на названии (ОАО "ДВЭУК") или же ввести подзаголовок в этом месте.

Второй крупный блок должен начинаться на "ПОВЕСТКА ДНЯ". Проверим.

In [ ]:
df_b2 = df_protokol.loc[(df['Tokens'] == "ПОВЕСТКА")]
df_b2

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
105198,protokol_2_vosa.html.txt,ПОВЕСТКА,O,O,O,O,O,3_Блок_Повестка,O,O,O


В а других документах такой же номер?

In [ ]:
df_b2 = df.loc[(df['Tokens'] == "ПОВЕСТКА")]
df_b2

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
574,Протокол ВОСА № 2 от 19_12_2013.html.txt,ПОВЕСТКА,O,O,O,O,O,3_Блок_Повестка,O,O,O
3676,protocol_gosa_18.01.2019.html.txt,ПОВЕСТКА,O,O,O,O,O,3_Блок_Повестка,O,O,O
15108,protocol_31.05.2019.html.txt,ПОВЕСТКА,O,O,O,O,O,3_Блок_Повестка,O,O,O
22725,vbank_protocol25_vosa_070318.html.txt,ПОВЕСТКА,O,O,O,O,O,3_Блок_Повестка,O,O,O
24226,Протокол ВОСА № 2 от 26_12_2017.html.txt,ПОВЕСТКА,O,O,O,O,O,3_Блок_Повестка,O,O,O
26309,protocol_meeting_07032018.html.txt,ПОВЕСТКА,O,O,O,O,O,3_Блок_Повестка,O,O,O
28127,vbank_protocol_vosa_070318.html.txt,ПОВЕСТКА,O,O,O,O,O,3_Блок_Повестка,O,O,O
29527,Протокол ВОСА № 1 от 18_10_2013.html.txt,ПОВЕСТКА,O,O,O,O,O,3_Блок_Повестка,O,O,O
39176,Protocol BOCA_18_06_2018.html.txt,ПОВЕСТКА,O,O,O,O,O,3_Блок_Повестка,O,O,O
44297,protokol_gosa_30 06 2011.html.txt,ПОВЕСТКА,O,O,O,O,O,3_Блок_Повестка,O,O,O


Да, нормер такой же.

Предыдущий блок выделен персоналиям и начинается с "Председательствующий"

In [ ]:
df_b2 = df_protokol.loc[(df['Tokens'] == "Председательствующий")]
df_b2

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
104959,protokol_2_vosa.html.txt,Председательствующий,O,O,B-Должность,O,O,2_Блок_Участники,O,O,O
108343,protokol_2_vosa.html.txt,Председательствующий,O,O,B-Должность,O,O,4_Блок_Обсуждение,O,O,O


In [ ]:
df_protokol.loc[104959]

DocName     protokol_2_vosa.html.txt
Tokens          Председательствующий
KE_0                               O
KE_1                               O
KE_2                     B-Должность
KE_3                               O
KE_4                               O
Blocks_0            2_Блок_Участники
Blocks_1                           O
Blocks_2                           O
Blocks_3                           O
Name: 104959, dtype: object

Действительно, предыдущий токен относится к верному блоку. Везде ли так?

In [ ]:
df_b2 = df.loc[(df['Tokens'] == "Председательствующий")]
df_b2

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
199,Протокол ВОСА № 2 от 19_12_2013.html.txt,Председательствующий,O,O,B-Должность,O,O,2_Блок_Участники,O,O,O
1294,Протокол ВОСА № 2 от 19_12_2013.html.txt,Председательствующий,O,O,B-Должность,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,O
2871,Протокол ВОСА № 2 от 19_12_2013.html.txt,Председательствующий,O,O,B-Должность,O,O,4_Блок_Обсуждение,O,O,O
2897,Протокол ВОСА № 2 от 19_12_2013.html.txt,Председательствующий,O,O,B-Должность,O,O,10_Блок_Подписи,O,O,O
5394,protocol_gosa_18.01.2019.html.txt,Председательствующий,O,O,B-Должность,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,I-4_1_ОбсуждениеПовестки->Выслушали_Обсудили
...,...,...,...,...,...,...,...,...,...,...,...
108343,protokol_2_vosa.html.txt,Председательствующий,O,O,B-Должность,O,O,4_Блок_Обсуждение,O,O,O
108626,GOSA17_06_2014.html.txt,Председательствующий,O,O,B-Должность,O,O,2_Блок_Участники,O,O,O
111967,GOSA17_06_2014.html.txt,Председательствующий,O,O,B-Должность,O,O,4_Блок_Обсуждение,O,4_1_ОбсуждениеПовестки,O
113890,GOSA17_06_2014.html.txt,Председательствующий,O,O,B-Должность,O,O,4_Блок_Обсуждение,O,O,O


C учётом того, что в протоколах в нескольких местах встречаются персоналии, этот блок тоже верный.